In [2]:
import pickle
import itertools
import open3d as o3d
import numpy as np

In [3]:
with open("48_args.curves", "rb") as pickle_file:
    curve_fragment = pickle.load(pickle_file)

In [4]:
mesh = o3d.io.read_triangle_mesh("plant3.ply")
edges = mesh.get_non_manifold_edges(allow_boundary_edges=False)

In [5]:
def edges_to_lineset(mesh, edges, color):
    ls = o3d.geometry.LineSet()
    ls.points = mesh.vertices
    ls.lines = edges
    colors = np.empty((np.asarray(edges).shape[0], 3))
    colors[:] = color
    ls.colors = o3d.utility.Vector3dVector(colors)
    return ls

In [6]:
def rotation_mat(angle):
    Rx = np.array([[1,0,0],
                 [0, np.cos(angle[0]), -np.sin(angle[0])],
                 [0, np.sin(angle[0]), np.cos(angle[0])]])

    Ry = np.array([[np.cos(angle[1]), 0, np.sin(angle[1])],
                 [0,1,0],
                 [-np.sin(angle[1]), 0, np.cos(angle[1])]])

    Rz = np.array([[np.cos(angle[2]), -np.sin(angle[2]), 0],
                 [np.sin(angle[2]), np.cos(angle[2]), 0],
                 [0,0,1]])
    return Rz@Rx@Ry


In [50]:
geoms = [mesh]
geoms.append(edges_to_lineset(mesh, edges, (1, 0, 0)))

In [51]:
#o3d.visualization.draw_geometries(geoms)

In [52]:
#angle = [np.pi/2, -np.pi/2, np.pi/2] # pcd.rotate()
angle = [-np.pi/2, np.pi/2, np.pi/2]
R = rotation_mat(angle)
R_mirror = np.array([[-1,0,0],
                    [0,1,0],
                    [0,0,1]])

In [53]:
flatten_curve_frag = np.array(list(itertools.chain.from_iterable(curve_fragment)))
flatten_curve_frag = (np.linalg.inv(R_mirror)@np.linalg.inv(R)@flatten_curve_frag.T).T

point_num = flatten_curve_frag.shape[0]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(flatten_curve_frag)
pcd.colors = o3d.utility.Vector3dVector(np.repeat(np.array((0, 1, 0)).reshape(1,-1), point_num, axis=0))
#pcd.rotate(R_mirror,(0,0,0))
#pcd.rotate(R,(0,0,0))
pcd.scale(10,(0,0,0))
geoms.append(pcd)

In [54]:
o3d.visualization.draw_geometries(geoms)

In [13]:
np.asarray(pcd.points)

array([[-0.07272509, -0.78590161,  3.87891568],
       [-0.07274501, -0.7773458 ,  3.87893287],
       [-0.10040024, -0.76781375,  3.88996609],
       ...,
       [-0.15372069, -1.72558039,  4.63847169],
       [-0.15419293, -1.72505646,  4.63771314],
       [-0.1546651 , -1.72453261,  4.63695474]])